In [196]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error

In [209]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

In [23]:
%%bash
for i in `seq 2 80`; do 
    cat hw3-data/my_train.csv | cut -f $i -d ',' | sort | uniq | wc -l; 
done | awk '{s+=$1-1} END {print s}'

7227


In [24]:
%%bash
header=$(head -n 1 hw3-data/my_train.csv)
for i in $(seq 2 80); do
    col_name=$(echo "$header" | cut -f $i -d ',')
    count=$(cut -f $i -d ',' hw3-data/my_train.csv | sort | uniq | wc -l)
    echo "$col_name: $count"
done

MSSubClass:       16
MSZoning:        6
LotFrontage:      109
LotArea:      990
Street:        3
Alley:        4
LotShape:        5
LandContour:        5
Utilities:        3
LotConfig:        6
LandSlope:        4
Neighborhood:       26
Condition1:       10
Condition2:        9
BldgType:        6
HouseStyle:        9
OverallQual:       11
OverallCond:       10
YearBuilt:      111
YearRemodAdd:       62
RoofStyle:        7
RoofMatl:        9
Exterior1st:       16
Exterior2nd:       17
MasVnrType:        6
MasVnrArea:      306
ExterQual:        5
ExterCond:        6
Foundation:        7
BsmtQual:        6
BsmtCond:        6
BsmtExposure:        6
BsmtFinType1:        8
BsmtFinSF1:      602
BsmtFinType2:        8
BsmtFinSF2:      132
BsmtUnfSF:      731
TotalBsmtSF:      687
Heating:        7
HeatingQC:        5
CentralAir:        3
Electrical:        7
1stFlrSF:      722
2ndFlrSF:      391
LowQualFinSF:       22
GrLivArea:      811
BsmtFullBath:        5
BsmtHalfBath:        4
FullBath: 

In [291]:
column_names = pd.read_csv("hw3-data/my_train.csv", nrows=1).columns.tolist()

train_data = pd.read_csv("hw3-data/my_train.csv", sep=",", skiprows=1, names=column_names, engine="python")
dev_data = pd.read_csv("hw3-data/my_dev.csv", sep=",", skiprows=1, names=column_names, engine="python")


In [292]:
test_column_names = pd.read_csv("hw3-data/test.csv", nrows=1).columns.tolist()

test_data = pd.read_csv("hw3-data/test.csv", sep=",", skiprows=1, names=test_column_names, engine="python")

In [293]:
# Separate features and target variable
X_train = train_data[column_names[1:-1]]  # Features
y_train = np.log(train_data['SalePrice'])  # Target variable

X_dev = dev_data[column_names[1:-1]]  # Features
y_dev = np.log(dev_data['SalePrice'])  # Target variable

In [294]:
X_test = test_data[test_column_names[1:]]

In [295]:
# cat_cols = [
#     'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
#     'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
#     'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
#     'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
#     'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
#     'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
#     'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
# ]
# 'BsmtFullBath', 'BsmtHalfBath', 'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces', 'GarageCars', 'MoSold', 'YrSold'
num_cols = [
    'LotFrontage', 'LotArea', 'YearBuilt',
    'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 
    'GarageYrBlt', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 
]

cat_cols = np.array([c for c in column_names[1:-1] if c not in num_cols])



In [269]:
X_train[num_cols] = X_train[num_cols].fillna(0)
X_dev[num_cols] = X_dev[num_cols].fillna(0)

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_81091/2435914748.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[num_cols] = X_train[num_cols].fillna(0)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_81091/2435914748.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_dev[num_cols] = X_dev[num_cols].fillna(0)


In [270]:
X_train[cat_cols] = X_train[cat_cols].astype(str)
X_dev[cat_cols] = X_dev[cat_cols].astype(str)

/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_81091/1122748261.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_train[cat_cols] = X_train[cat_cols].astype(str)
/var/folders/3h/5ch9gkv90lxdnr54086zz5rh0000gn/T/ipykernel_81091/1122748261.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_dev[cat_cols] = X_dev[cat_cols].astype(str)


In [271]:
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

# encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# combined_data = pd.concat([X_train, X_dev], axis=0)

# combined_binary_data = encoder.fit_transform(combined_data)

# # Binarization
# # X_train_binary = encoder.transform(X_train)
# # X_dev_binary = encoder.transform(X_dev)

# X_train_binary = combined_binary_data[:len(X_train)]
# X_dev_binary = combined_binary_data[len(X_train):]


In [272]:
# num_processor = MinMaxScaler(feature_range=(0, 2))
# num_processor = 'passthrough'
num_processor = MinMaxScaler()
cat_processor = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

In [273]:
preprocessor = ColumnTransformer([
    ('cat', cat_processor, cat_cols),
    ('num', num_processor, num_cols)
])

In [274]:
smart_X_train = preprocessor.fit_transform(X_train)
smart_X_dev = preprocessor.transform(X_dev)

In [290]:
len(preprocessor.get_feature_names_out())

386

In [275]:
# Train linear regression model
model = LinearRegression()
model.fit(smart_X_train, y_train)

LinearRegression()

In [276]:
# Make predictions
y_pred = model.predict(smart_X_dev)

In [296]:
new_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Binarization
X_train_binary = new_encoder.fit_transform(X_train.astype(str))
X_dev_binary = new_encoder.transform(X_dev.astype(str))



In [289]:
len(new_encoder.get_feature_names_out())

7226

In [297]:
X_test_binary = new_encoder.transform(X_test.astype(str))

In [298]:
# Train linear regression model
model = LinearRegression()
model.fit(X_train_binary, y_train)

LinearRegression()

In [299]:
# Make predictions
y_pred = model.predict(X_dev_binary)

In [300]:
np.sqrt(mean_squared_error(y_dev, y_pred))


0.15233237053005208

In [197]:
# Get feature names from one-hot encoder
feature_names = new_encoder.get_feature_names_out()

# Create a DataFrame with feature names and their coefficients
coef_df = pd.DataFrame({'Feature': feature_names, 'Coefficient': model.coef_})

# Sort the DataFrame by coefficients in descending order
sorted_coef_df = coef_df.sort_values(by='Coefficient', ascending=False)

# Top 10 most positive features
top_positive_features = sorted_coef_df.head(10)

# Top 10 most negative features
top_negative_features = sorted_coef_df.tail(10)

# Print the results
print("Top 10 Most Positive Features:")
print(top_positive_features)

print("\nTop 10 Most Negative Features:")
print(top_negative_features)


Top 10 Most Positive Features:
                   Feature  Coefficient
5900            FullBath_3     0.139237
1204         OverallQual_9     0.137293
1162  Neighborhood_StoneBr     0.125342
4712          2ndFlrSF_472     0.113064
1203         OverallQual_8     0.106477
1398      RoofMatl_WdShngl     0.092519
5283        GrLivArea_1192     0.089511
1155  Neighborhood_NoRidge     0.087755
6060          GarageCars_3     0.086133
391           LotArea_8029     0.085403

Top 10 Most Negative Features:
                Feature  Coefficient
1195      OverallQual_1    -0.089186
218        LotArea_5000    -0.090997
6058       GarageCars_1    -0.094239
1207      OverallCond_3    -0.101207
420        LotArea_8281    -0.107380
2443     BsmtFinSF2_311    -0.108756
1198      OverallQual_3    -0.113082
7040  EnclosedPorch_236    -0.123418
5165      GrLivArea_968    -0.126604
15     MSZoning_C (all)    -0.204339


In [198]:
intercept = model.intercept_

In [59]:
# Top 10 Most Positive Features
top_positive_features.to_csv('top_positive_features.csv', index=False)

# Top 10 Most Negative Features
top_negative_features.to_csv('top_negative_features.csv', index=False)


In [199]:
intercept

12.172652588014603

In [301]:
# y_pred_test = np.round(model.predict(X_test_binary), decimals=3)
y_pred_test = model.predict(X_test_binary)

In [302]:
y_pred_test = np.exp(y_pred_test)

In [303]:
# Assigning new predicted column to the dataset
test_data['SalePrice'] = y_pred_test

# Create a new DataFrame with 'ID' and 'SalePrice'
result_df = test_data[['Id', 'SalePrice']]

# Save the DataFrame to a CSV file with a header
result_df.to_csv('naive_binarization.csv', index=False)

In [97]:
# num_processor = 'passthrough'
num_processor = MinMaxScaler(feature_range=(0, 2))
cat_processor = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

cat_cols = [
    'MSSubClass', 'MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig',
    'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'OverallQual', 'OverallCond',
    'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual',
    'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1',
    'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
    'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
    'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition'
]
num_cols = [
    'LotFrontage', 'LotArea', 'YearBuilt',
    'YearRemodAdd', 'MasVnrArea', 'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF', 'TotalBsmtSF',
    '1stFlrSF', '2ndFlrSF', 'LowQualFinSF', 'GrLivArea', 'BsmtFullBath', 'BsmtHalfBath',
    'FullBath', 'HalfBath', 'BedroomAbvGr', 'KitchenAbvGr', 'TotRmsAbvGrd', 'Fireplaces',
    'GarageYrBlt', 'GarageCars', 'GarageArea', 'WoodDeckSF', 'OpenPorchSF', 'EnclosedPorch',
    '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'MoSold', 'YrSold'
]

preprocessor = ColumnTransformer([
    ('cat', cat_processor, cat_cols),
    ('num', num_processor, num_cols)
])

In [98]:
naive_X_train = preprocessor.fit_transform(X_train.astype(str))
naive_X_dev = preprocessor.transform(X_dev.astype(str))

In [99]:
# Train linear regression model
model = LinearRegression()
model.fit(naive_X_train, y_train)

ValueError: Input X contains NaN.
LinearRegression does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [94]:
# Make predictions
y_pred = model.predict(naive_X_dev)
np.sqrt(mean_squared_log_error(y_dev, y_pred))

0.16993639439412808